In [1]:
import torchvision
from glob import glob
from torch.utils.data import Dataset

import torch
import torch.nn as nn
import numpy as np
import pandas as pd

### Dataset class for getting positive and negative classes for training

In [2]:
from torch.utils.data import Dataset
from torchvision import transforms
import random
from PIL import Image

class SmileDataset(Dataset):
    def __init__(self, relations, person_to_image) -> None:
        super().__init__()

        self.relations = relations
        self.person_to_image = person_to_image
        self.people_labels = list(person_to_image.keys())
    
    def __len__(self):
        #2 times since we only have positive examples and every alternate example is negative
        return len(self.relations) * 2 

    def __getitem__(self, index):

        #positive example
        if (index%2==0): 
            p1, p2 = self.relations[index//2]
            label = 1

        #negative example
        else:
            while True:
                p1 = random.choice(self.people_labels) 
                p2 = random.choice(self.people_labels) 
                if (p1 != p2) and (p1, p2) not in self.relations and (p2, p1) not in self.relations:
                    break 
            label = 0

        path1 = self.person_to_image[p1]
        path2 = self.person_to_image[p2]
        
        image1 = Image.open(path1[random.randint(0, len(path1)-1)])
        image2 = Image.open(path2[random.randint(0, len(path2)-1)])
        
        convert_image_to_tensor_transform = transforms.ToTensor()

        image1_tensor = convert_image_to_tensor_transform(image1)
        image2_tensor = convert_image_to_tensor_transform(image2)

        return image1_tensor, image2_tensor, label

### Creating a dictionary with (Family) \\\\ (ID) as the key and the path to the images under that one person

In [3]:
import pandas as pd

train_file_path = "./train_relationships.csv"
train_images_path = "./train/"

all_images = glob(train_images_path + "*/*/*.jpg")
train_person_to_images = {}

for x in all_images:

    if x.split("\\")[-3] + "/" + x.split("\\")[-2] not in train_person_to_images:
        train_person_to_images[x.split("\\")[-3] + "/" + x.split("\\")[-2]] = [x]

    else:
        train_person_to_images[x.split("\\")[-3] + "/" + x.split("\\")[-2]].append(x)

train_people = [x.split("\\")[-3] + "/" + x.split("\\")[-2] for x in all_images]
train_people = list(dict.fromkeys(train_people)) # removing the duplicates

relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in train_people and x[1] in train_people] #Check if people are in the training dataset


### Instantiate the SmileDataset class

In [4]:
from torch.utils.data import DataLoader

train_dataset = SmileDataset(relations = relationships, person_to_image= train_person_to_images)
trainloader = DataLoader(train_dataset, batch_size= 100, shuffle = True)

### Training time!! :) (Work in progress)

In [5]:
from SiameseNet import SiameseNet

lr = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SiameseNet().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params= model.parameters(), lr = lr)

num_epoch = 100
best_epoch = 0

history = []
accuracy = []

for epoch in range(num_epoch):
    train_loss = 0.0
    running_loss = 0.0
    running_corrects = 0
    for i, batch in enumerate(trainloader):
        optimizer.zero_grad()
        
        tensor1, tensor2, label = batch
        tensor1, tensor2, label = tensor1.to(device), tensor2.to(device), label.float().view(-1,1).to(device)
        output = model(tensor1, tensor2)

        preds = output>0.5
        
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        running_loss += loss.item()
        running_corrects += torch.sum(preds == label)
        
        step = 10
        if i % step == step-1:
            print(' [{} - {:.2f}%],\ttrain loss: {:.5}'.format(epoch+1, 100*(i+1)/len(trainloader), running_loss/step/200))
            running_loss = 0
        
    

c:\Users\ianch\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 [1 - 14.71%],	train loss: 0.003624
 [1 - 29.41%],	train loss: 0.0034453


KeyboardInterrupt: 